<a href="https://colab.research.google.com/github/LazyTurtleknight/deepl/blob/main/Testing_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Skip this cell if you are not working with colab

!pip install opendatasets
!pip install pandas
!pip install albumentations

# Download dataset from kaggle
od.download(
    "https://www.kaggle.com/datasets/balraj98/deepglobe-land-cover-classification-dataset")


In [48]:
# Define constants

# directories
data_dir = 'deepglobe-land-cover-classification-dataset' # change to directory containing the data
train_dir = 'train'

In [49]:
import opendatasets as od
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import albumentations as alb
import cv2

from PIL import Image

In [50]:
# Load metadata and get random sample
metadata = pd.read_csv(os.path.join(data_dir, 'metadata.csv'))

sample = metadata[metadata['split'] == 'train'].sample(n=1)

In [ ]:
# Plot random sample

def plot(sample):

  plt.figure(figsize=(5,4))
  ax = plt.subplot(2,2,1)
  plt.imshow(np.asarray(Image.open(os.path.join(data_dir, sample['sat_image_path'].iloc[0]))))
  plt.gray()
  ax.get_yaxis().set_visible(False)
  ax.get_xaxis().set_visible(False)

  ax = plt.subplot(2,2,2)
  plt.imshow(np.asarray(Image.open(os.path.join(data_dir, sample['mask_path'].iloc[0]))))
  plt.gray()
  ax.get_yaxis().set_visible(False)
  ax.get_xaxis().set_visible(False)

  plt.show()

plot(sample)

In [59]:
# Test albumentations

sample_path = os.path.join(data_dir, sample['sat_image_path'].iloc[0])
sample_mask_path = os.path.join(data_dir, sample['mask_path'].iloc[0])

# Define augmentation pipeline
transform = alb.Compose([
    alb.RandomCrop(width=256, height=256),
    alb.HorizontalFlip(p=0.5),
    alb.RandomBrightnessContrast(p=0.2),],
    # we want the mask and the image to have the same augmentation (or at least the same crop)
    # this way we pass the image and the mask simultaneously to the pipeline
    additional_targets={'image': 'image', 'mask': 'mask'}
    )

# Read an image with OpenCV and convert it to the RGB colorspace
image = cv2.imread(sample_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image_mask = cv2.imread(sample_mask_path)
image_mask = cv2.cvtColor(image_mask, cv2.COLOR_BGR2RGB)

# Augment an image
transformed = transform(image = image, mask = image_mask)
transformed_image = transformed['image']
transformed_image_mask = transformed['mask']

In [ ]:
plt.subplot(121),plt.imshow(transformed_image),plt.title('Image');
plt.subplot(122),plt.imshow(transformed_image_mask),plt.title('Mask');